<a href="https://colab.research.google.com/github/jacdiste/NeuralTennis/blob/main/NeuralTennis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **NeuralTennis**

In [1]:
!pip install --quiet torch torchvision opencv-python-headless matplotlib plotly albumentations scikit-learn ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 125.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 84.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 60.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 37.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 110.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 61.6 MB/s eta 0:00:00


In [2]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [3]:
import os

BASE_DIR = "/content/drive/MyDrive/NeuralTennis"
for sub in ["input", "models", "output"]:
    os.makedirs(f"{BASE_DIR}/{sub}", exist_ok=True)

In [4]:
import cv2

video_path = f"{BASE_DIR}/input/4.mp4"
cap = cv2.VideoCapture(video_path)
if not cap.isOpened():
    raise IOError(f"Cannot open {video_path}")
ret, frame = cap.read()
cap.release()
if not ret:
    raise IOError("Cannot read the first frame")
print("Video loaded – resolution:", frame.shape[1], "×", frame.shape[0])

Video loaded – resolution: 1920 × 1080


In [5]:
import cv2

def read_video(video_path):
    cap = cv2.VideoCapture(video_path)
    frames = []
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        frames.append(frame)
    cap.release()
    return frames

def save_video(output_video_frames, output_video_path):
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_video_path, fourcc, 60, (output_video_frames[0].shape[1], output_video_frames[0].shape[0]))
    for frame in output_video_frames:
        out.write(frame)
    out.release()
    print(f"Video saved to {output_video_path}")


In [7]:
from ultralytics import YOLO

!yolo task=detect mode=train model=yolov5x.pt data=/content/drive/MyDrive/NeuralTennis/data/data.yaml epochs=100 imgsz=640 batch=8

PRO TIP 💡 Replace 'model=yolov5x.pt' with new 'model=yolov5xu.pt'.
YOLOv5 'u' models are trained with https://github.com/ultralytics/ultralytics and feature improved performance vs standard YOLOv5 models trained with https://github.com/ultralytics/yolov5.

Ultralytics 8.3.166 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (NVIDIA L4, 22693MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/drive/MyDrive/NeuralTennis/data/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf

In [ ]:
import torch
import torch.nn as nn
import torchvision.models as models

class CourtKeypointNet(nn.Module):
    def __init__(self, n_keypoints=14):
        super().__init__()
        # Encoder: ResNet34 già pre-allenata su ImageNet
        #  - estraiamo i blocchi conv1…layer4 come feature extractor
        resnet = models.resnet34(pretrained=True)
        self.enc1 = nn.Sequential(resnet.conv1, resnet.bn1, resnet.relu)  # 64 × H/2
        self.enc2 = nn.Sequential(resnet.maxpool, resnet.layer1)          # 64 × H/4
        self.enc3 = resnet.layer2                                         # 128 × H/8
        self.enc4 = resnet.layer3                                         # 256 × H/16
        self.enc5 = resnet.layer4                                         # 512 × H/32

        # Decoder: upsample + conv per ricostruire risoluzione
        def upconv(in_ch, out_ch):
            return nn.Sequential(
                nn.ConvTranspose2d(in_ch, out_ch, kernel_size=2, stride=2),
                nn.BatchNorm2d(out_ch),
                nn.ReLU(inplace=True),
            )

        self.up5 = upconv(512, 256)  # → 256 × H/16
        self.up4 = upconv(256, 128)  # → 128 × H/8
        self.up3 = upconv(128,  64)  # →  64 × H/4
        self.up2 = upconv( 64,  32)  # →  32 × H/2

        # Head finale: da 32 canali a N heatmap, uno per keypoint
        self.head = nn.Conv2d(32, n_keypoints, kernel_size=1)


In [ ]:
import torch, torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import cv2, numpy as np

class KeypointDataset(Dataset):
    def __init__(self, img_dir, annot_csv, img_size=480):
        # annot_csv contiene filename + 28 colonne x1,y1,…,x14,y14 (normali o assolute)
        self.df = pd.read_csv(annot_csv)
        self.img_dir = img_dir
        self.img_size = img_size

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        # 1) carica immagine e ridimensiona
        img = cv2.imread(f"{self.img_dir}/{row.filename}")
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = cv2.resize(img, (self.img_size, self.img_size))
        img_t = torch.from_numpy(img).permute(2,0,1).float()/255.0

        # 2) genera heatmap target per ciascun keypoint
        H = W = self.img_size
        n_kp = 14
        hmaps = np.zeros((n_kp, H, W), dtype=np.float32)
        for i in range(n_kp):
            # recupera x,y (in pixel o normalizzati)
            x = row[f"x{i+1}"] * W   # se normalizzati tra 0 e 1
            y = row[f"y{i+1}"] * H
            xv, yv = np.meshgrid(np.arange(W), np.arange(H))
            # Gaussiana centrata sul punto, sigma=5px
            hmaps[i] = np.exp(-((xv-x)**2 + (yv-y)**2)/(2*5**2))
        return img_t, torch.from_numpy(hmaps)



model = CourtKeypointNet(n_keypoints=14).cuda()
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3)
criterion = nn.MSELoss()  # misura la distanza tra heatmap predetta e target

In [40]:
#!pip install ultralytics
from ultralytics import YOLO
import cv2
import pandas as pd
import numpy as np
import math


class PlayerTracker:
    def __init__(self,model_path):
        self.model = YOLO(model_path)

    def detect_frame(self,frame):
        results = self.model.track(frame, persist=True)[0]
        id_name_dict = results.names

        player_dict = {}
        for box in results.boxes:
            track_id = int(box.id.tolist()[0])
            result = box.xyxy.tolist()[0]
            object_cls_id = box.cls.tolist()[0]
            object_cls_name = id_name_dict[object_cls_id]
            if object_cls_name == "person":
                player_dict[track_id] = result

        return player_dict

    def draw_bboxes(self,video_frames, player_detections):
        output_video_frames = []
        for frame, player_dict in zip(video_frames, player_detections):
            # Draw Bounding Boxes
            for track_id, bbox in player_dict.items():
                x1, y1, x2, y2 = bbox
                cv2.putText(frame, f"Player ID: {track_id}",(int(bbox[0]),int(bbox[1] -10 )),cv2.FONT_HERSHEY_SIMPLEX, 0.9, (197, 197, 197), 2)
                cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (197, 197, 197), 2)
            output_video_frames.append(frame)

        return output_video_frames


class BallTracker:
    def __init__(self,model_path):
        self.model = YOLO(model_path)

    def interpolate_ball_positions(self, ball_positions):
        ball_positions = [x.get(1,[]) for x in ball_positions]

        df_ball_positions = pd.DataFrame(ball_positions,columns=['x1','y1','x2','y2'])

        df_ball_positions = df_ball_positions.interpolate()
        df_ball_positions = df_ball_positions.bfill()

        ball_positions = [{1:x} for x in df_ball_positions.to_numpy().tolist()]

        return ball_positions


    def detect_frame(self,frame):
        results = self.model.predict(frame,conf=0.7,imgsz=1600, save=True)[0] ##save

        ball_dict = {}
        for box in results.boxes:
            result = box.xyxy.tolist()[0]
            ball_dict[1] = result

        return ball_dict

    def draw_bboxes(self,video_frames, player_detections):
        output_video_frames = []
        for frame, ball_dict in zip(video_frames, player_detections):
            # Draw Bounding Boxes
            for track_id, bbox in ball_dict.items():
                x1, y1, x2, y2 = bbox
                cv2.putText(frame, f"Ball ID: {track_id}",(int(bbox[0]),int(bbox[1] -10 )),cv2.FONT_HERSHEY_SIMPLEX, 0.9, (96, 255, 168), 2)
                cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (96, 255, 168), 2)
            output_video_frames.append(frame)

        return output_video_frames

In [41]:
def main():

    input_video_path = "/content/drive/MyDrive/NeuralTennis/input/1.mp4"
    video_frames = read_video(input_video_path)

    player_tracker = PlayerTracker("/content/drive/MyDrive/NeuralTennis/models/yolov8x.pt")
    player_detections = [player_tracker.detect_frame(frame) for frame in video_frames]
    video_frames = player_tracker.draw_bboxes(video_frames, player_detections)

    ball_tracker = BallTracker("/content/runs/detect/train2/weights/best.pt")
    ball_detections = [ball_tracker.detect_frame(frame) for frame in video_frames]
    ball_detections = ball_tracker.interpolate_ball_positions(ball_detections)
    video_frames = ball_tracker.draw_bboxes(video_frames, ball_detections)

    save_video(video_frames, "/content/drive/MyDrive/NeuralTennis/output/1dckck.mp4")

if __name__ == "__main__":
    main()

Output streaming troncato alle ultime 5000 righe.
0: 384x640 11 persons, 3 chairs, 2 clocks, 14.1ms
Speed: 2.4ms preprocess, 14.1ms inference, 1.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 11 persons, 3 chairs, 2 clocks, 13.3ms
Speed: 3.4ms preprocess, 13.3ms inference, 1.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 11 persons, 3 chairs, 2 clocks, 13.3ms
Speed: 2.3ms preprocess, 13.3ms inference, 1.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 11 persons, 3 chairs, 2 clocks, 13.3ms
Speed: 2.3ms preprocess, 13.3ms inference, 1.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 11 persons, 3 chairs, 2 clocks, 13.5ms
Speed: 2.4ms preprocess, 13.5ms inference, 1.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 11 persons, 3 chairs, 2 clocks, 13.6ms
Speed: 3.4ms preprocess, 13.6ms inference, 1.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 11 persons, 3 chairs, 2 clocks, 13.4ms
Speed: 2.4ms